# Using GraphRAG-SDK to Create a Multi-Agent-Orchestrator RAG System based on Knowledge Graph from Structured Data

GraphRAG-SDK provides a powerful tool, enhanced by LLM technology, for building a Multi-Agent-Orchestrator RAG System. This example demonstrates how to create a knowledge graph-based multi-agent system from structured data and how to use the orchestrator to manage the agents and perform question answering.

In [ ]:
!pip install graphrag_sdk

In [1]:

from json import loads
from dotenv import load_dotenv
from graphrag_sdk.orchestrator import Orchestrator
from graphrag_sdk.agents.kg_agent import KGAgent
from graphrag_sdk.models.openai import OpenAiGenerativeModel
from graphrag_sdk import (
    Ontology, Entity, Relation, Attribute, AttributeType, KnowledgeGraph, KnowledgeGraphModelConfig
)

# Load environment variables
load_dotenv()

# Configuration
# OPENAI_API_KEY = "sk-"# OpenAI API key

True

### Ontology - Predefined Ontology

We will begin with an initial ontology for the agents, based on the knowledge that interests us.


In [2]:
# Restaurants Ontology
restaurants_ontology = Ontology()

# Attractions Ontology
attractions_ontology = Ontology()

# Manually created Ontology by adding entities and relations
restaurants_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="Restaurant",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="rating",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
            Attribute(
                name="food_type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Restaurant",
        target="City",
    )
)

attractions_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="Attraction",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Attraction",
        target="City",
    )
)

### KG from Structure Data

Creating the KG from the data that will be used by the agents

In [3]:
# Define the model
model = OpenAiGenerativeModel("gpt-4o")

# Create the KG from the predefined ontology.
# Restaurants KG
restaurants_kg = KnowledgeGraph(
    name="restaurants",
    ontology=restaurants_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)

# Attractions KG
attractions_kg = KnowledgeGraph(
    name="attractions",
    ontology=attractions_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)

# Add nodes and edges to the KGs from the data
with open("data/cities.json") as f:
    cities = loads(f.read())
with open("data/restaurants.json") as f:
    restaurants = loads(f.read())
with open("data/attractions.json") as f:
    attractions = loads(f.read())

for city in cities:
    restaurants_kg.add_node(
        "City",
        {
            "name": city["name"],
            "weather": city["weather"],
            "population": city["population"],
        },
    )
    restaurants_kg.add_node("Country", {"name": city["country"]})
    restaurants_kg.add_edge(
        "IN_COUNTRY",
        "City",
        "Country",
        {"name": city["name"]},
        {"name": city["country"]},
    )

    attractions_kg.add_node(
        "City",
        {
            "name": city["name"],
            "weather": city["weather"],
            "population": city["population"],
        },
    )
    attractions_kg.add_node("Country", {"name": city["country"]})
    attractions_kg.add_edge(
        "IN_COUNTRY",
        "City",
        "Country",
        {"name": city["name"]},
        {"name": city["country"]},
    )

for restaurant in restaurants:
    restaurants_kg.add_node(
        "Restaurant",
        {
            "name": restaurant["name"],
            "description": restaurant["description"],
            "rating": restaurant["rating"],
            "food_type": restaurant["food_type"],
        },
    )
    restaurants_kg.add_edge(
        "IN_CITY",
        "Restaurant",
        "City",
        {"name": restaurant["name"]},
        {"name": restaurant["city"]},
    )

for attraction in attractions:
    attractions_kg.add_node(
        "Attraction",
        {
            "name": attraction["name"],
            "description": attraction["description"],
            "type": attraction["type"],
        },
    )
    attractions_kg.add_edge(
        "IN_CITY",
        "Attraction",
        "City",
        {"name": attraction["name"]},
        {"name": attraction["city"]},
    )

### Agents

Creating the Agents based on the KG.

In [4]:
# The following agent is specialized in finding restaurants.
restaurants_agent = KGAgent(
    agent_id="restaurants_agent",
    kg=restaurants_kg,
    introduction="I'm a restaurant agent, specialized in finding the best restaurants for you.",
)

# The following agent is specialized in finding tourist attractions.
attractions_agent = KGAgent(
    agent_id="attractions_agent",
    kg=attractions_kg,
    introduction="I'm an attractions agent, specialized in finding the best attractions for you.",
)

### Orchestrator - Multi-Agent System

The orchestrator manages the usage of agents and handles questioning. Each agent is registered with the orchestrator, which determines how to engage the agents to provide the best possible answer when a question is asked.

In [5]:

# Initialize the orchestrator and register agents
orchestrator = Orchestrator(
    model,
    backstory="You are a trip planner, and you want to provide the best possible itinerary for your clients.",
)
orchestrator.register_agent(restaurants_agent)
orchestrator.register_agent(attractions_agent)

# Query the orchestrator
runner = orchestrator.ask("Create a two-day itinerary for a trip to Rome. Please don't ask me any questions; just provide the best itinerary you can.")

In [6]:
print(runner.output)

Thank you for your question about creating a two-day itinerary for a trip to Rome. Here is a detailed itinerary that covers key attractions and recommended dining options, thoughtfully curated to give you the best experience during your stay.

### Day 1 Itinerary

**Morning:**

1. **Colosseum**: Begin your adventure with a visit to the Colosseum, an ancient amphitheater renowned for gladiatorial contests and public spectacles. It is one of Rome's most iconic landmarks, offering an incredible glimpse into the city’s historical grandeur.

**Mid-Morning:**

2. **Vatican Museums**: Next, head to the Vatican Museums, a complex of fascinating museums and galleries showcasing remarkable works of art collected by Popes over centuries. This is a must-see for art and history enthusiasts.

**Afternoon:**

3. **Trevi Fountain**: In the afternoon, visit the Trevi Fountain, a magnificent Baroque fountain famous for its impressive sculptures and the tradition of tossing coins. Relax and enjoy the bea